In [ ]:
import os
import re
import pandas as pd
import numpy as np
from selenium import webdriver as wd

pd.set_option("display.max_columns", 500)
pd.set_option("display.width", 1000)

path = os.getcwd()
driver = wd.Chrome(path + '/chromedriver')

# DataFrame for result
blurb_df = pd.DataFrame(columns = ['name', 'blurb', 'state', 'category', 'funding_rate', 'pledged', 'goal', 'launched', 'deadline'])

def webcrawl(start, end):
    global blurb_df
    for page in range(start,end+1):
        print("==================================")
        print("Collecting page",page,"......")
        print("==================================")

        # load page
        driver.get('https://www.kicktraq.com/dayones/?page={}'.format(page))

        # get list of all project list and info
        prj_list = driver.find_elements_by_xpath("//div[@class='project-infobox']")
        prj_length = len(prj_list)

        for i in range(prj_length):
            x = prj_list[i]

            ## TITLE
            name = x.find_element_by_xpath("h2").text

            ## BLURB
            blurb = x.find_element_by_xpath("div[not(@class)]").text

            ## CATEGORY
            cat =  x.find_element_by_xpath("div[@class='project-cat']")
            cat_lst = cat.find_elements_by_tag_name("a")
            category = cat_lst[0].text

            ## PROJECT INFORMATION
            info = x.find_element_by_xpath("div[@class='project-infobits']/div[@class='project-details']")
            prj_info = info.text
            # finding rate
            rate_info = info.find_element_by_tag_name("span").text
            percent = int(re.findall('\d+', rate_info)[0])

            if percent >= 100:
                pledged = prj_info.split('\n')[1].split(': ')[1].split(' of ')[0]
                goal = prj_info.split('\n')[1].split(': ')[1].split(' of ')[1].split(' (')[0]
                year = prj_info.split('\n')[3].split('(')[1][:-1]
                launched = prj_info.split('\n')[3].split(": ")[1].split(' -> ')[0]
                deadline = prj_info.split('\n')[3].split(": ")[1].split(' -> ')[1].split(' (')[0]
                row = [name, blurb, 'success', category, percent, pledged, goal, year + ' ' + launched, year + ' ' + deadline]
                blurb_df = blurb_df.append(pd.Series(row, index=['name', 'blurb', 'state', 'category', 'funding_rate', 'pledged', 'goal', 'launched', 'deadline']), ignore_index=True)
